# Web Scraping de Datos de la NBA

Este notebook extrae estadísticas de la NBA desde Basketball-Reference.com para los años 1991-2024.

## Fuentes de Datos
Extraemos tres conjuntos de datos principales:
1. **Datos de Votación del MVP**: Candidatos al MVP y resultados de votación por temporada
2. **Estadísticas de Jugadores**: Estadísticas por partido para todos los jugadores de la NBA
3. **Estadísticas de Equipos**: Clasificaciones y métricas de rendimiento de equipos

Todos los datos provienen de [Basketball-Reference.com](https://www.basketball-reference.com/)

In [1]:
# Importar bibliotecas necesarias para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import pandas as pd
import cloudscraper
import time

## 1. Configuración Inicial

Definir el rango de años a extraer (1991-2024)

In [2]:
# Definir el rango de años para extraer datos de la NBA
years = list(range(1991, 2025))

## 2. Extracción de Datos de Votación del MVP

### Qué extraemos:
Cada fila representa un candidato al MVP en un año específico con sus resultados de votación y estadísticas.

### Columnas clave:
- **Rank**: Posición en la votación del MVP (puede tener sufijos como "T" para empates, ej: "9T")
- **Player**: Nombre del jugador
- **Age**: Edad del jugador durante esa temporada
- **Tm**: Abreviatura del equipo
- **First**: Número de votos de primer lugar recibidos
- **Pts Won, Pts Max, Share**: Métricas de puntuación de votación
- **G**: Partidos jugados
- **MP**: Minutos jugados por partido
- **PTS, TRB, AST, STL, BLK**: Puntos, rebotes, asistencias, robos y tapones
- **FG%, 3P%, FT%**: Porcentajes de tiros de campo, triples y tiros libres
- **WS, WS/48**: Win Shares y Win Shares por 48 minutos
- **Year**: Año de la temporada (añadido por nuestro script)

### Nota:
- Algunas celdas contienen valores vacíos (ej: 3P% para jugadores interiores que no tiran triples)

In [3]:
# Plantilla de URL para páginas de votación del MVP
url_nba_start = "https://www.basketball-reference.com/awards/awards_{}.html"

In [36]:
# Crear una sesión de cloudscraper para evitar la protección de Cloudflare
# cloudscraper.create_scraper() devuelve una sesión similar a requests.Session
# que automáticamente resuelve los desafíos de Cloudflare
scraper = cloudscraper.create_scraper()

# Descargar y guardar páginas HTML para cada año
for year in years:
    # Formatear la URL con el año actual
    url_nba = url_nba_start.format(year)
    
    # Realizar petición HTTP para obtener la página
    data = scraper.get(url_nba)
    time.sleep(2)
    # Guardar el archivo HTML localmente (modo w+ crea el archivo si no existe)
    with open("mvps_data/{}.html".format(year), "w+") as file:
        file.write(data.text)

In [5]:
# Probar leyendo el archivo de datos del MVP de 1991
with open("mvps_data/1991.html") as file:
    page = file.read()

In [6]:
# Parsear el contenido HTML usando BeautifulSoup
# Crea una estructura de árbol navegable desde el string HTML
soup = BeautifulSoup(page, "html.parser")

In [7]:
# Eliminar la fila de encabezado innecesaria si existe
over_header = soup.find("tr", class_="over_header")
if over_header:
    over_header.decompose()

In [8]:
# Extraer solo la tabla de votación del MVP de la página
mvp_table = soup.find(id="mvp")

In [9]:
# Convertir la tabla HTML a un DataFrame de pandas
# pd.read_html() devuelve una lista de DataFrames, tomamos el primero
mvp_1991 = pd.read_html(str(mvp_table))
mvp_1991[0]

/tmp/ipykernel_5537/3989412907.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp_1991 = pd.read_html(str(mvp_table))


,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,1,Michael Jordan,27,CHI,77,891,960,0.928,82,37.0,31.5,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321
1,2,Magic Johnson,31,LAL,10,497,960,0.518,79,37.1,19.4,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251
2,3,David Robinson,25,SAS,6,476,960,0.496,82,37.7,25.6,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264
3,4,Charles Barkley,27,PHI,2,222,960,0.231,67,37.3,27.6,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258
4,5,Karl Malone,27,UTA,0,142,960,0.148,82,40.3,29.0,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225
5,6,Clyde Drexler,28,POR,1,75,960,0.078,82,34.8,21.5,6.7,6.0,1.8,0.7,0.482,0.319,0.794,12.4,0.209
6,7,Kevin Johnson,24,PHO,0,32,960,0.033,77,36.0,22.2,3.5,10.1,2.1,0.1,0.516,0.205,0.843,12.7,0.220
7,8,Dominique Wilkins,31,ATL,0,29,960,0.030,81,38.0,25.9,9.0,3.3,1.5,0.8,0.470,0.341,0.829,11.4,0.177
8,9T,Larry Bird,34,BOS,0,25,960,0.026,60,38.0,19.4,8.5,7.2,1.8,1.0,0.454,0.389,0.891,6.6,0.140
9,9T,Terry Porter,27,POR,0,25,960,0.026,81,32.9,17.0,3.5,8.0,2.0,0.1,0.515,0.415,0.823,13.0,0.235


In [10]:
# Procesar todos los años y combinar en un solo conjunto de datos
dfs = []

for year in years:
    # Leer el archivo HTML guardado
    with open(f"mvps_data/{year}.html".format(year)) as file:
        page = file.read()
        soup = BeautifulSoup(page, "html.parser")
        
    # Eliminar fila de encabezado innecesaria
    over_header = soup.find("tr", class_="over_header")
    if over_header:
        over_header.decompose()
    
    # Extraer tabla del MVP y convertir a DataFrame
    mvp_table = soup.find(id="mvp")
    mvp = pd.read_html(str(mvp_table))[0]
    
    # Añadir una columna para rastrear el año
    mvp["Year"] = year
    
    dfs.append(mvp)

/tmp/ipykernel_5537/3079911020.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]
/tmp/ipykernel_5537/3079911020.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]
/tmp/ipykernel_5537/3079911020.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]
/tmp/ipykernel_5537/3079911020.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]
/tmp/ipykern

In [11]:
# Concatenar todos los DataFrames anuales en un solo conjunto de datos
mvps = pd.concat(dfs)

In [12]:
# Mostrar el conjunto de datos combinado de MVP
mvps

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Michael Jordan,27,CHI,77,891,960,0.928,82,37.0,...,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
1,2,Magic Johnson,31,LAL,10,497,960,0.518,79,37.1,...,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1991
2,3,David Robinson,25,SAS,6,476,960,0.496,82,37.7,...,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991
3,4,Charles Barkley,27,PHI,2,222,960,0.231,67,37.3,...,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258,1991
4,5,Karl Malone,27,UTA,0,142,960,0.148,82,40.3,...,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,5,Jalen Brunson,27,NYK,0,142,990,0.143,77,35.4,...,3.6,6.7,0.9,0.2,0.479,0.401,0.847,11.2,0.198,2024
5,6,Jayson Tatum,25,BOS,0,86,990,0.087,74,35.7,...,8.1,4.9,1.0,0.6,0.471,0.376,0.833,10.4,0.189,2024
6,7,Anthony Edwards,22,MIN,0,18,990,0.018,79,35.1,...,5.4,5.1,1.3,0.5,0.461,0.357,0.836,7.5,0.130,2024
7,8,Domantas Sabonis,27,SAC,0,3,990,0.003,82,35.7,...,13.7,8.2,0.9,0.6,0.594,0.379,0.704,12.6,0.206,2024


In [13]:
# Exportar a archivo CSV
mvps.to_csv("mvps.csv")

## 3. Extracción de Estadísticas de Jugadores

### Desafío:
La página de estadísticas de jugadores usa JavaScript para renderizar la tabla dinámicamente. Esto significa que las peticiones HTTP simples no capturarán la tabla completa - necesitamos Selenium para simular el comportamiento del navegador y hacer scroll para cargar todo el contenido.

In [14]:
# Plantilla de URL para estadísticas por partido de jugadores
# Nota: Esta página usa renderizado con JavaScript, requiere Selenium
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [15]:
# Importar Selenium para automatización del navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Configurar la ruta del ChromeDriver
# Esto le dice a Selenium dónde encontrar el ejecutable de ChromeDriver
ruta_servicio = Service('/home/jpgso/Desktop/Proyect DAI/NBA Predictions/web_scraping/chromedriver-linux64/chromedriver')

# Inicializar navegador Chrome con el driver especificado
driver = webdriver.Chrome(service=ruta_servicio)

In [ ]:
# Extraer estadísticas de jugadores para todos los años usando Selenium
for year in years:
    
    url = player_stats_url.format(year)

    # Abrir la URL en el navegador controlado por Selenium
    driver.get(url)
    
    # Hacer scroll hasta el final para activar la carga perezosa de todo el contenido de la tabla
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Esperar a que se cargue el contenido
    time.sleep(3)

    # Obtener el HTML completamente renderizado
    page_html = driver.page_source
    
    # Guardar archivo HTML localmente (w+ crea el archivo si no existe)
    with open("players_data/{}.html".format(year), "w+") as file:
        file.write(page_html)

In [16]:
# Parsear archivos HTML guardados de estadísticas de jugadores
dfs = []

for year in years:
    # Leer el archivo HTML guardado
    with open("players_data/{}.html".format(year)) as file:
        page_html = file.read()
        soup = BeautifulSoup(page_html, "html.parser")

    # Eliminar filas de encabezado repetidas que aparecen en medio de la tabla
    over_header = soup.find("tr", class_="thead")
    if over_header:
        over_header.decompose()
        
    # Extraer tabla de estadísticas de jugadores
    player_table = soup.find(id="per_game_stats")
    player = pd.read_html(str(player_table))[0]
    
    # Añadir columna de año para rastreo
    player["Year"] = year
    
    dfs.append(player)

/tmp/ipykernel_5537/1217456396.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player = pd.read_html(str(player_table))[0]
/tmp/ipykernel_5537/1217456396.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player = pd.read_html(str(player_table))[0]
/tmp/ipykernel_5537/1217456396.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player = pd.read_html(str(player_table))[0]
/tmp/ipykernel_5537/1217456396.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player = pd.read_html(str(player

In [17]:
# Combinar todos los datos de jugadores en un solo DataFrame
players = pd.concat(dfs)

In [18]:
# Eliminar la columna 'Awards' ya que no será necesaria para la predicción
if "Awards" in players.columns:
    del players["Awards"]
players.columns

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Year'],
      dtype='object')

In [19]:
# Mostrar las estadísticas combinadas de jugadores
players

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Michael Jordan,27,CHI,SG,82,82,37.0,12.1,22.4,...,1.4,4.6,6.0,5.5,2.7,1.0,2.5,2.8,31.5,1991
1,2,Karl Malone,27,UTA,PF,82,82,40.3,10.3,19.6,...,2.9,8.9,11.8,3.3,1.1,1.0,3.0,3.3,29.0,1991
2,3,Bernard King,34,WSB,SF,64,64,37.5,11.1,23.6,...,1.8,3.2,5.0,4.6,0.9,0.3,4.0,2.9,28.4,1991
3,4,Charles Barkley,27,PHI,SF,67,67,37.3,9.9,17.4,...,3.9,6.3,10.1,4.2,1.6,0.5,3.1,2.6,27.6,1991
4,5,Patrick Ewing,28,NYK,C,81,81,38.3,10.4,20.3,...,2.4,8.8,11.2,3.0,1.0,3.2,3.6,3.5,26.6,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,569,Ron Harper Jr.,23,TOR,PF,1,0,4.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,2024
755,570,Justin Jackson,28,MIN,SF,2,0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024
756,571,Dmytro Skapintsev,25,NYK,C,2,0,1.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024
757,572,Javonte Smart,24,PHI,PG,1,0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024


In [20]:
# Exportar estadísticas de jugadores a CSV
players.to_csv("players.csv")

## 4. Extracción de Estadísticas de Equipos

Extraer clasificaciones y datos de rendimiento de equipos para ambas conferencias Este y Oeste.

In [21]:
# Plantilla de URL para clasificaciones de equipos
url_teams = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [ ]:
# Extraer clasificaciones de equipos para todos los años
for year in years:
    
    url = url_teams.format(year)

    # Abrir URL en navegador controlado por Selenium
    driver.get(url)
    
    # Hacer scroll para cargar todo el contenido dinámico
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

    # Obtener HTML completamente renderizado
    page_html = driver.page_source
    
    # Guardar archivo HTML localmente
    with open("team_data/{}.html".format(year), "w+") as file:
        file.write(page_html)

In [22]:
# Parsear HTML de clasificaciones de equipos para ambas conferencias Este y Oeste
dfs = []

for year in years:
    # Leer archivo HTML guardado
    with open("team_data/{}.html".format(year)) as file:
        page_html = file.read()
        soup = BeautifulSoup(page_html, "html.parser")

    # Eliminar filas de encabezado repetidas
    over_header = soup.find("tr", class_="thead")
    if over_header:
        over_header.decompose()
        
    # Extraer clasificación de la Conferencia Este
    team_table = soup.find(id="divs_standings_E")
    team = pd.read_html(str(team_table))[0]
    
    # Añadir columna de año
    team["Year"] = year
    
    # Renombrar columna de conferencia a genérico "Team" para consistencia
    team["Team"] = team["Eastern Conference"]
    del team["Eastern Conference"]
    
    dfs.append(team)
    
    # Procesar clasificación de la Conferencia Oeste
    soup = BeautifulSoup(page_html, "html.parser")
    over_header = soup.find("tr", class_="thead")
    if over_header:
        over_header.decompose()
        
    team_table = soup.find(id="divs_standings_W")
    team = pd.read_html(str(team_table))[0]
    
    # Añadir columna de año
    team["Year"] = year
    
    # Renombrar columna de conferencia a genérico "Team"
    team["Team"] = team["Western Conference"]
    del team["Western Conference"]
    
    dfs.append(team)

/tmp/ipykernel_5537/218457996.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team = pd.read_html(str(team_table))[0]
/tmp/ipykernel_5537/218457996.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team = pd.read_html(str(team_table))[0]
/tmp/ipykernel_5537/218457996.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team = pd.read_html(str(team_table))[0]
/tmp/ipykernel_5537/218457996.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team = pd.read_html(str(team_table))[0]
/tmp/ipy

In [23]:
# Combinar datos de equipos de todos los años y ambas conferencias
team = pd.concat(dfs)

In [24]:
# Mostrar estadísticas combinadas de equipos
team

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
13,50,32,.610,—,117.9,115.6,2.30,2024,Dallas Mavericks*
14,49,33,.598,1.0,115.1,110.7,4.46,2024,New Orleans Pelicans*
15,41,41,.500,9.0,114.3,113.2,1.24,2024,Houston Rockets
16,27,55,.329,23.0,105.8,112.8,-6.57,2024,Memphis Grizzlies


In [25]:
# Exportar estadísticas de equipos a CSV
team.to_csv("teams.csv")